In [3]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))


In [14]:
from joblib import load, dump
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
from pathlib import Path
import sys

# Add project root to Python path
ROOT_DIR = Path.cwd().parents[1]  # adjust depending on where your notebook lives
sys.path.append(str(ROOT_DIR))

from joblib import load
MODEL_PATH = ROOT_DIR / "app" / "ml" / "models" / "intent_pipeline.joblib"
bundle = load(MODEL_PATH)

MODEL_DIR = ROOT_DIR / "app" / "ml" / "models"
MODEL_DIR.mkdir(parents=True, exist_ok=True)  # ensure it exists


# 1️⃣ Load your current pipeline

pipeline = bundle["pipeline"] if isinstance(bundle, dict) else bundle
label_names = bundle.get("label_names") if isinstance(bundle, dict) else None

# 2️⃣ Prepare your data again (reuse the same dataset you trained with)
import pandas as pd
df = pd.read_csv("dataset.csv")

X = df["text"]
y = df["intent"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# 3️⃣ Extract the underlying classifier
clf = pipeline.named_steps["clf"]

# 4️⃣ Wrap it with calibration
calibrated_clf = CalibratedClassifierCV(clf, cv=5, method="sigmoid")

# 5️⃣ Fit on transformed features
X_train_vec = pipeline.named_steps["feat"].transform(X_train)
calibrated_clf.fit(X_train_vec, y_train)

# 6️⃣ Evaluate
X_test_vec = pipeline.named_steps["feat"].transform(X_test)
y_pred = calibrated_clf.predict(X_test_vec)
y_pred_prob = calibrated_clf.predict_proba(X_test_vec)

print("✅ Calibrated Model Report:\n")
print(classification_report(y_test, y_pred))

# 7️⃣ Replace old classifier in pipeline
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

new_pipeline = Pipeline([
    ("prep", pipeline.named_steps["prep"]),
    ("feat", pipeline.named_steps["feat"]),
    ("clf", calibrated_clf)
])


MODEL_DIR = ROOT_DIR / "app" / "ml" / "models"
MODEL_DIR.mkdir(parents=True, exist_ok=True)  # ensure it exists
MODEL_PATH = MODEL_DIR / "intent_pipeline_calibrated.joblib"
# 8️⃣ Save back to disk
dump(
    {"pipeline": new_pipeline, "label_names": label_names},
    MODEL_PATH
)

print("\n🚀 Saved calibrated model -> app/models/intent_pipeline_calibrated.joblib")


C:\Users\Ayman\Desktop\projects\harri-ai-interview-task\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\Ayman\Desktop\projects\harri-ai-interview-task\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\Ayman\Desktop\projects\harri-ai-interview-task\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\Ayman\Desktop\proje

✅ Calibrated Model Report:

                    precision    recall  f1-score   support

code_review_policy       1.00      1.00      1.00        19
deployment_history       0.85      1.00      0.92        17
deployment_process       0.94      0.84      0.89        19
     dev_env_setup       0.89      1.00      0.94        17
    employees_info       0.76      0.72      0.74        18
 escalation_policy       0.95      1.00      0.97        19
jira_ticket_status       1.00      1.00      1.00        18
  onboarding_guide       1.00      0.89      0.94        19
      out_of_scope       1.00      0.88      0.93        16
    team_structure       0.80      0.84      0.82        19

          accuracy                           0.92       181
         macro avg       0.92      0.92      0.92       181
      weighted avg       0.92      0.92      0.92       181


🚀 Saved calibrated model -> app/models/intent_pipeline_calibrated.joblib
